# vLife Virtusa
## Predicting Frequent Visit to Pharma
### Usecase Description
_Using mathematical approach, we are predicting whether a patient frequently visiting to pharmacy or not.We have used marketscan dataset for this approach._


### Dataset Source
Data for this usecase is extracted from Data as Service available in vLife Innovation Platform.

### Dataset Description
##### Features Available

* enrolid
* svcdate  
* daysupp

### Import Section

In [1]:
import pandas as pd
import numpy as np
from datetime import timedelta
import warnings
warnings.filterwarnings("ignore")

In [2]:
df=pd.read_csv('TwentyID.csv')

In [3]:
pd.set_option('display.max_columns',None,'display.max_rows',None)

In [4]:
len(df)

1153

In [5]:
df.head()

,enrolid,svcdate,daysupp
0,147601,01/19/2016,0
1,147601,01/29/2016,0
2,147601,02/17/2016,0
3,147601,02/17/2016,0
4,147601,02/17/2016,0


### Algorithm to Predict visit to Pharma

> Helper function to predict frequent visit to Pharma

In [6]:
def patodrugs(df):
    pdcval = []
    mprval = []
    countfalse = 0
    data = df.drop_duplicates(['svcdate'])
    dat = len(data)
    data['Date'] = pd.to_datetime(df['svcdate'])
    data.sort_values(by='Date', ascending=True,inplace = True)
    temp = df['daysupp'].apply(np.ceil).apply(lambda x: pd.Timedelta(x, unit='D'))
    data['RX End Date'] = data['Date'] + temp
    data['Rx End Date'] = pd.DatetimeIndex(data['RX End Date']) - pd.DateOffset(1)
    data['overlap'] = (data['RX End Date'].shift()-data['Date']) > timedelta(0)
    data.reset_index(drop=True, inplace=True)
    for i in data['overlap']:
        if i==False:
            countfalse=countfalse+1
    if dat != countfalse:
        val1 = []
        val2 = []
        count = 0
        for i in data['overlap']:
            if i==False:
                val1.append(None)
                val2.append(None)
            else:
                if data['overlap'][count-1] == False:
                    val1.append(data['Rx End Date'][count-1] + pd.DateOffset(1))
                    val2.append(val1[-1] + pd.DateOffset(data['daysupp'][count]-1))
                else:
                    val1.append(val2[-1] + pd.DateOffset(1))  
                    val2.append(val1[-1] + pd.DateOffset(data['daysupp'][count]-1))
            count = count+1
        adjstartDate = pd.DataFrame({'Rx Adj Start date':val1})
        adjendDate = pd.DataFrame({'Rx Adj End date':val2})
        adjustedData = pd.concat([adjstartDate, adjendDate], axis=1)
        FinalData = pd.concat([data, adjustedData], axis=1)
        FinalData.drop(['RX End Date','svcdate'], axis=1,inplace = True)
        FinalData.rename(index=str, columns={"Date": "svcdate"})
        FinalData['date']=pd.to_datetime(FinalData['Rx Adj End date'])
        FinalData['year'] = FinalData['date'].dt.year
        temp = 0
        temp1 = 0
        finalyear = pd.DataFrame(FinalData['year'].notnull())
        finalyear
        for x in finalyear['year']:
            if x == True:
                temp = FinalData['year'][temp1]
                break
            temp1 = temp1 +1
    
        count1 = 0;
        l = []
        l2 = []
        import math
        x = float('nan')
        #math.isnan(x)
        type(x)
        for i in FinalData['year']:
            if temp < i :
                l.append(FinalData.iloc[FinalData.index[count1]])
                l2.append('2015-12-31')
                break
            else:
                l.append(FinalData.iloc[FinalData.index[count1]])
                l2.append(None)
            count1 = count1 + 1
        df = pd.DataFrame(l)
        df1=pd.DataFrame(l2)
        df1.rename(columns={0: "Reporting End Time"},inplace=True)
        pd.to_datetime(df1['Reporting End Time'])
        finaldata = pd.concat([df, df1], axis=1)

        repet = pd.to_datetime(finaldata['Reporting End Time'].iloc[finaldata.index[count1]])

        rasd = finaldata['Rx Adj Start date'].iloc[finaldata.index[count1]]

        datediff1 = repet - rasd
        datediff1.days
        adjustDaysCovered = datediff1.days +1

        st = finaldata['Date'].iloc[finaldata.index[count1]]

        datediff2 = repet - st
        datediff2.days
        WithoutAdjustment = datediff2.days +1

        l3=[]
        l4=[]
        for j in range(count1+1):
            if j == count1:
                l3.append(adjustDaysCovered)
                l4.append(WithoutAdjustment)
            else:
                l3.append(None)
                l4.append(None)

        df2=pd.DataFrame(l3)
        df2.rename(columns={0: "adjustDaysCovered"},inplace=True)
        df3=pd.DataFrame(l4)
        df3.rename(columns={0: "withoutAdjustment"},inplace=True)
        finaldata1 = pd.concat([df2, df3], axis=1)

        finaldata2 = pd.concat([finaldata, finaldata1], axis=1)


        StartDateOfReportingPeriod = finaldata2['Date'][0]


        EndDateOfReportingPeriod = pd.to_datetime(finaldata['Reporting End Time'].iloc[finaldata.index[count1]])


        val1 = finaldata2['daysupp'].sum()
        val2 = finaldata['daysupp'].iloc[finaldata.index[count1]]
        val3 = val1 - val2
        val4 = finaldata2['adjustDaysCovered'].iloc[finaldata2.index[count1]]
        val5 = val3 + val4
        DaysPatientIsCoveredWithMedicationAndWithAdjustment = val5.astype(int)

        val1 = finaldata2['daysupp'].sum()
        val2 = finaldata['daysupp'].iloc[finaldata.index[count1]]
        val3 = val1 - val2
        val4 = finaldata2['withoutAdjustment'].iloc[finaldata2.index[count1]]
        val5 = val3 + val4
        DaysPatientIsCoveredWithMedicationAndWithoutAdjustment = val5.astype(int)

        datediff3 = repet - StartDateOfReportingPeriod
        ReportingPeriod = datediff3.days + 1

        #PDC
        PDC = DaysPatientIsCoveredWithMedicationAndWithAdjustment/ReportingPeriod
        pdcad=PDC*100
        pdcval.append("{:.2%}".format(PDC))
        #MPR
        MPR = DaysPatientIsCoveredWithMedicationAndWithoutAdjustment/ReportingPeriod
        mprr=MPR*100
        mprval.append("{:.2%}".format(MPR)) 
        print(data)
        return pdcad,mprr
    else:
        return 0,0

### Main function
##### Function to get Prediction from Algorithm

In [7]:
if __name__ == "__main__":
    df = df.sort_values('enrolid',ascending=True)
    df.reset_index(drop=True)
    print(df['enrolid'].unique().tolist())
    names= int(input("Enter an id :"))
    dff = df.loc[df.enrolid==names]
    s,u = patodrugs(dff)
    print(s,u)
    if s >= 80:
        print("Given patient id is adherence to drugs")
    else:
        print("Given patient id is not adherence to drugs")

[16602, 20302, 28002, 35302, 44102, 58602, 68202, 80002, 125603, 126003, 127804, 138801, 142302, 147601, 148101, 157103, 166103, 174301, 184102, 200701]
Enter an id :68202
     enrolid     svcdate  daysupp       Date RX End Date Rx End Date  overlap
0      68202  01/05/2015       30 2015-01-05  2015-02-04  2015-02-03    False
1      68202  01/26/2015       30 2015-01-26  2015-02-25  2015-02-24     True
2      68202  01/27/2015       30 2015-01-27  2015-02-26  2015-02-25     True
3      68202  01/30/2015       30 2015-01-30  2015-03-01  2015-02-28     True
4      68202  02/03/2015       30 2015-02-03  2015-03-05  2015-03-04     True
5      68202  03/22/2015        7 2015-03-22  2015-03-29  2015-03-28    False
6      68202  03/23/2015       30 2015-03-23  2015-04-22  2015-04-21     True
7      68202  03/24/2015       30 2015-03-24  2015-04-23  2015-04-22     True
8      68202  03/28/2015       30 2015-03-28  2015-04-27  2015-04-26     True
9      68202  03/30/2015       28 2015-03-30  20

## END